<a href="https://colab.research.google.com/github/deitar/JiraWiki2Docx/blob/main/JiraWiki2Docx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xmltodict
!pip install htmldocx
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=adbd33b981cf24676474870fec9d8223f85aaf79497c7d3e0340b47465f048c4
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [3]:
import os
import re
import html
import json
import xmltodict
import requests
from datetime import datetime
from requests.auth import HTTPBasicAuth
from docx import Document
from htmldocx import HtmlToDocx
from docx.shared import Inches
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.shared import Inches, RGBColor, Pt, Cm
from docx.enum.text import WD_LINE_SPACING
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from itertools import groupby
from datetime import datetime
import numpy as np
import pandas as pd
from enum import Enum

In [4]:
sample_jira_text = '''h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity.

*Definition of Terms*

* Cash scarcity period: November 2022 to March 2023 (150 days)
** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)
* Period after cash scarcity: April 2023 to May 2023 (60 days)
**** Period after cash scarcity: April 2023 to May 2023 (60 days)

# test number 1
#* test number 11
#* test number 11
# test number 2
## test number 2

*Key Highlights:*

* Customers transactional behaviour before, during and after cash scarcity

||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarcity {color:#ff5630}AND {color}continued afterwards||
|Distinct Card Holders|3,674,893|{color:#ff5630}*1,310,454*{color}|602,235|

||* go
* be||
| |
| |


* *3.6 million (66%)* of the customers who completed card transactions during the cash scarcity period also carried out transactions prior to and after the cash scarcity period.
* *23% (1.31 million)* of the customers who carried out POS/WEB transactions during cash scarcity stopped card transactions at the end of the cash scarcity period
* Customers with POS/WEB transactions during cash scarcity but not after cash scarcity

||Period||Transacted prior to AND during cash scarcity BUT subsequently stopped||Transacted only during cash scarcity, NOT before or after cash scarcity||% of customers who transacted only during cash scarcity||
|Distinct Card Holders|{color:#36b37e}1,030,063 {color}|{color:#ff991f}280,391{color}|{color:#ff991f}21%{color}|

* *79% (1.03 of 1.31 million)* of these customers also carried out POS/WEB transactions in the 6 months prior to cash scarcity which means that they regularly transacted using their cards regardless of cash scarcity.-'''


In [5]:
class JiraWikiBlockTypes(Enum):
    LIST = 'list'
    HEADING = 'heading'
    TABLE = 'table'
    CODE = 'code'
    QUOTE = 'quote'
    PARAGRAPH = 'paragraph'

In [6]:
def detect_jira_block_type(jira_block):
    jira_block_trimmed = jira_block.strip()
    marker_heading = r'^h([1-6])\..*$'
    # marker_list = '^([\*|\-|\#]+)\s(.*)'
    marker_list = '^[*#-]+\s.*$'
    marker_table = '^[||]+.*$'
    if re.match(marker_heading, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.HEADING
    if re.match(marker_list, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.LIST
    if re.match(marker_table, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.TABLE
    return JiraWikiBlockTypes.PARAGRAPH

In [7]:
def get_jira_blocks(jira_text):
    return re.split('\n\n', jira_text)

In [8]:
def write_heading_jira_to_doc(text, doc_part_to_write_to):
    pattern_heading = r'^h([1-6])\.(.*)'
    heading_match = re.match(pattern_heading, text)
    heading_number, heading_text = heading_match.groups()
    para = doc_part_to_write_to.add_paragraph()
    para.style = f'Heading {heading_number}'
    para.add_run(text=heading_text)

In [9]:
def write_list_jira_to_doc(text, doc_part_to_write_to):
    pattern_list_item = r'^([\*|\-|\#]+)\s{1}(.*)'
    # split block into lines
    lines = re.split('\n', text)
    for line in lines:
        list_type, list_text = re.match(pattern_list_item, line).groups()
        # issue 1: couldn't resolve the round and square bullets differently
        # issue 2: find a way of decorating nested lists with different point styles
        # issue 3: sublevel list have unwanted top space to their parent list
        max_list_level = 3
        style_name = "List Number" if list_type[:max_list_level][-1] == '#' else 'List Bullet'
        style_name = f'{style_name} {str(min(len(list_type), max_list_level)) if len(list_type) > 1 else ""}'.strip()
        para = doc_part_to_write_to.add_paragraph()
        para.style = style_name
        para.add_run(text = list_text)

In [10]:
def write_generic_jira_to_doc(text, doc_part_to_write_to):
    para = doc_part_to_write_to.add_paragraph()
    para.add_run(text = text)

In [11]:
def write_jira_block_to_doc(jira_block):

    # handle leading new lines
    new_lines, actual_jira_block = re.match('(\n*)(.*)', jira_block, re.DOTALL).groups()
    num_new_lines = new_lines.count('\n')

    # add new lines by occurence
    for i in range(num_new_lines):
        para = doc.add_paragraph()
        para.add_run()

    # 1. heading
    if detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.HEADING:
        write_heading_jira_to_doc(actual_jira_block, doc)
    # 2. list
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.LIST:
        write_list_jira_to_doc(actual_jira_block, doc)
    # 3. table
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.TABLE:
        write_generic_jira_to_doc(actual_jira_block, doc)

    # any other
    else:
        write_generic_jira_to_doc(actual_jira_block, doc)

        # handle text effects



In [11]:
# Initialize a new Document
doc = Document()

In [12]:
for block in get_jira_blocks(sample_jira_text):
    print(repr(block))
    write_jira_block_to_doc(block)

'h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity.'
'*Definition of Terms*'
'* Cash scarcity period: November 2022 to March 2023 (150 days)\n** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)\n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n**** Period after cash scarcity: April 2023 to May 2023 (60 days)'
'# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2'
'*Key Highlights:*'
'* Customers transactional behaviour before, during and after cash scarcity'
'||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarci

In [13]:
doc.save('output.docx')

In [28]:
# jira_blocks = sample_jira_text.split('\n\n')
# # lines
# print(len(jira_blocks))

In [98]:
# test_block = '\n\n* *3.6 million (66%)* of the +to+ and after the cash scarcity period.\n* *23% (1.31 million)* ed out POS/WEB transactions d'
test_block = '_3.6_ {{mil_lion_ (66%)}} of the to *and* after _the_ {color:green}cash scarcity{color} period.\n'
# text_strong_matches = re.findall('\*\S[^*]*\S\*', test_block, re.DOTALL)
# print(text_strong_matches)
pattern_strong_effect = '((\*)(\S[^*]*\S)(\*))'
pattern_italics_effect = '((\_)(\S[^_]*\S)(\_))'
pattern_color_effect = '((\{color:[A-Za-z]*\})(\S((?!{color}).)*\S)(\{color\}))'
pattern_text_effect = re.compile(f'{pattern_strong_effect}|{pattern_italics_effect}|{pattern_color_effect}', re.IGNORECASE | re.DOTALL)
for m in pattern_text_effect.finditer(test_block):
    # print(m.start(), '\t', m.groups(), '\t', m.end(), '\t') # test_block[m.start()])
    print(list(filter(lambda x: x, m.groups())))
# for italic in

# print(re.split('\*\S[^*]*\S\*', test_block, re.DOTALL))

  # write_jira_block_to_doc(block)
# also consider nested markups e.g text in bold and inside strike through

['_3.6_', '_', '3.6', '_']
['_lion_', '_', 'lion', '_']
['*and*', '*', 'and', '*']
['_the_', '_', 'the', '_']
['{color:green}cash scarcity{color}', '{color:green}', 'cash scarcity', 't', '{color}']


In [79]:
test_block = '_3.6_ *mil_lion_ (66%)* of the to *and* after _the_ cash scarcity period.\n'

text_effect_para_dict = {}

def effecter(text, existing_effect=None):
  global text_effect_para_dict
  # asterisk/markup with no space after start or before end
  pattern_strong_effect = '((\*)(\S[^*]*\S)(\*))'
  pattern_italics_effect = '((\_)(\S[^_]*\S)(\_))'
  pattern_color_effect = '((\{color:[A-Za-z]*\})(\S((?!{color}).)*\S)(\{color\}))'
  print(text)
  pattern_text_effect = re.compile(f'{pattern_strong_effect}|{pattern_italics_effect}|{pattern_color_effect}', re.DOTALL|re.IGNORECASE)
  # if not pattern_text_effect.match(text):
  #     return
  for m in pattern_text_effect.finditer(text):
      full_match, text_effect_ope_tag, inner_text, text_effect_cls_tag = tuple(filter(lambda x: x, m.groups()))
      # print(full_match, '\t', text_effect_ope_tag, '\t', inner_text, '\t', text_effect_cls_tag)
      # text_effect_markup = test_block[m.start()]
      # inner_text = m.group().strip(text_effect_markup)
      text_effect_para_dict[m.start()] = inner_text
      effecter(inner_text)

effecter(test_block)

_3.6_ *mil_lion_ (66%)* of the to *and* after _the_ cash scarcity period.

3.6
mil_lion_ (66%)
lion
and
the


In [78]:
text_effect_para_dict

{0: '3.6', 6: 'mil_lion_ (66%)', 3: 'lion', 34: 'and', 46: 'the'}

In [33]:
re.split('\n{2}', sample_jira_text)

['h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity.',
 '*Definition of Terms*',
 '* Cash scarcity period: November 2022 to March 2023 (150 days)\n** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)\n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n**** Period after cash scarcity: April 2023 to May 2023 (60 days)',
 '# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2',
 '*Key Highlights:*',
 '* Customers transactional behaviour before, during and after cash scarcity',
 '||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting durin

In [ ]:

tl = '\n# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2'
pattern_list = r'(\n)*([\*|\-|\#]+)\s{1}(.*?)(?:$|\n)'
re.findall(pattern_list, tl)

In [ ]:
re.match('^[*#-]+.*$', '''# test number 1
#* test number 11
#* test number 11
# test number 2
## test number 2''', re.DOTALL)

In [ ]:
table_header_pattern = r'[^\|]+(?=\|\|)'

In [ ]:
import re

text = '''||Heading 1
* first
* second||Heading 2||
|* Item 1
* Item 2
* Item 3|# Item 1
# Item 2
# Item 3|'''

# regex_pattern = r'[\|\|]+([^\|]*)[\|\|]+'
regex_pattern = r'\|\|.*\|\|'

matches = re.findall(regex_pattern, sample_jira_text, re.DOTALL)
print("Captured text:", matches)

Captured text: ['||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarcity {color:#ff5630}AND {color}continued afterwards||\n|Distinct Card Holders|3,674,893|{color:#ff5630}*1,310,454*{color}|602,235|\n\n||* go\n* be||\n| |\n| |\n\n\n* *3.6 million (66%)* of the customers who completed card transactions during the cash scarcity period also carried out transactions prior to and after the cash scarcity period. \n* *23% (1.31 million)* of the customers who carried out POS/WEB transactions during cash scarcity stopped card transactions at the end of the cash scarcity period\n* Customers with POS/WEB transactions during cash scarcity but not after cash scarcity\n\n||Period||Transacted prior to AND during cash scarcity BUT subsequently stopped||Transacted only during cash scarcity, NOT before or after cash scarcity||% o

In [ ]:
import re

text = '''||Heading 1||Heading 2||
|* Item 1
* Item 2
* Item 3|# Item 1
# Item 2
# Item 3|'''

regex_pattern = r'\|\|([^|]+)\|\|\n((?:.*\n)*)'

match = re.search(regex_pattern, text)
if match:
    captured_heading1 = match.group(1)
    captured_heading2 = match.group(2)
    print("Heading 1:", captured_heading1)
    print("Heading 2:\n", captured_heading2)
else:
    print("No match found")

Heading 1: Heading 2
Heading 2:
 |* Item 1
* Item 2
* Item 3|# Item 1
# Item 2



In [ ]:
sample_jira_text

'h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity. \n\n*Definition of Terms*\n\n* Cash scarcity period: November 2022 to March 2023 (150 days) \n** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days) \n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n**** Period after cash scarcity: April 2023 to May 2023 (60 days)\n\n# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2\n\n*Key Highlights:*\n\n* Customers transactional behaviour before, during and after cash scarcity\n\n||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during ca

In [ ]:
lines = sample_jira_text.split('\n')

In [ ]:
lines

['h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity. ',
 '',
 '*Definition of Terms*',
 '',
 '* Cash scarcity period: November 2022 to March 2023 (150 days) ',
 '** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days) ',
 '* Period after cash scarcity: April 2023 to May 2023 (60 days)',
 '**** Period after cash scarcity: April 2023 to May 2023 (60 days)',
 '',
 '# test number 1',
 '#* test number 11',
 '#* test number 11',
 '# test number 2',
 '## test number 2',
 '',
 '*Key Highlights:*',
 '',
 '* Customers transactional behaviour before, during and after cash scarcity',
 '',
 '||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash sca

In [ ]:
cur_line = lines[0]
# print(cur_line)
pattern_heading = r'^h([1-6])\.(.*)'
re.match(pattern_heading, cur_line).groups()

('2',
 ' -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity. ')

In [ ]:
line = lines[9]
print(line)
pattern_list = r'^([\*]+|[\-]+|[\#]+)\s{1}(.*)'
re.findall(pattern_list, '### test number 1')#.groups()

# test number 1


[('###', 'test number 1')]

In [ ]:
re.findall(r'^([\*]+|[\-]+)\s{1}(.*)', '-- Cash scarcity period: November 2022 to March 2023 (150 days) ')

[('--', 'Cash scarcity period: November 2022 to March 2023 (150 days) ')]

In [ ]:
# re.match(r'(\|\|[^\|]*\|\|)', '||heading 1||heading 2||heading 3||').groups()
re.match(r'\|\|([^\|]*)\|\|', '||heading 1||heading 2||heading 3||').groups()
re.findall(r'([^\|]+)[\|\|]+', '||heading 1|heading 2||heading 3||')
# re.findall(r'[^\|]+(?=\|\|)', '||heading 1||heading 2||heading 3||')

['heading 1', 'heading 2', 'heading 3']

In [ ]:
re.split(r'[\|\|]+', '|heading 1|heading 2||heading 3||')

['', 'heading 1', 'heading 2', 'heading 3', '']

In [ ]:
import re

text = '||heading 1||heading 2||heading 3||'

regex_pattern = r'\|\|(.*?)\|\|'

matches = re.findall(regex_pattern, text)
print("Captured text:", matches)


Captured text: ['heading 1', 'heading 3']


In [ ]:
# Initialize a new Document
doc = Document()
body_table = doc.add_table(rows=1, cols=1)
container = body_table.rows[0].cells[0]

def convert_jira_to_docx(jira_text):

    # Split the Jira text into separate lines
    lines = jira_text.split('\n')

    cur_table = None

    # Process each line of Jira text
    for line_pos, line in enumerate(lines):
        # if it's the first line take the existing paragraph, otherwise, create new one
        # para = doc.paragraphs[0] if line_pos == 0 else doc.add_paragraph()
        para = container.add_paragraph()

        # Skip empty lines
        if not line.strip():
            para.add_run()
            continue

        # Check for different Jira elements

        # regex patterns
        pattern_heading = r'^h([1-6])\.(.*)'
        pattern_list = r'^([\*|\-|\#]+)\s{1}(.*)'
        table_header_pattern = r'[^\|]+(?=\|\|)'

        # if wiki element is not a table
        if not line.startswith('|'):
            cur_table = None

        # 1. heading
        if re.match(pattern_heading, line):
            heading_match = re.match(pattern_heading, line)
            heading_number, heading_text = heading_match.groups()
            para.style = f'Heading {heading_number}'
            para.add_run(text=heading_text)
        # 2. list
        elif re.match(pattern_list, line):
            list_match = re.match(pattern_list, line)
            list_type, list_text = list_match.groups()
            # issue 1: couldn't resolve the round and square bullets differently
            # issue 2: find a way of decorating nested lists with different point styles
            # issue 3: sublevel list have unwanted top space to their parent list
            max_list_level = 3
            style_name = 'List Bullet'
            if list_type[:max_list_level][-1] == '#':
                style_name = "List Number"
            para.style = f'{style_name} {str(min(len(list_type), max_list_level)) if len(list_type) > 1 else ""}'.strip()
            para.add_run(text = list_text)
        # 3. table
        elif line.startswith('|'):
            header_texts = re.split(r'[\|\|]+', line)
            if header_texts:
              # header_texts = re.findall(table_header_pattern, line)
              n_cols = len(header_texts)

              # Create a table
              if not cur_table:
                  cur_table = doc.add_table(rows=1, cols=n_cols)
                  cur_table.style = 'Table Grid'
                  cur_row = cur_table.rows[0]
              else:
                  cur_row = cur_table.add_row()

              # populate header row
              heading_cells = cur_row.cells
              for i in range(len(heading_cells)):
                  heading_cells[i].text = header_texts[i]




        else:
            para.add_run(text=line)

    return doc

convert_jira_to_docx(sample_jira_text).save('output.docx')

In [ ]:
# Initialize a new Document
doc = Document()

def convert_jira_to_docx(jira_text):

    # Split the Jira text into separate lines
    lines = jira_text.split('\n')

    cur_table = None

    # Process each line of Jira text
    for line_pos, line in enumerate(lines):
        # if it's the first line take the existing paragraph, otherwise, create new one
        # para = doc.paragraphs[0] if line_pos == 0 else doc.add_paragraph()
        para = doc.add_paragraph()

        # Skip empty lines
        if not line.strip():
            para.add_run()
            continue

        # Check for different Jira elements

        # regex patterns
        pattern_heading = r'^h([1-6])\.(.*)'
        # pattern_list = r'^([\*]+|[\-]+|[\#]+)\s{1}(.*)'
        pattern_list = r'^([\*|\-|\#]+)\s{1}(.*)'
        table_header_pattern = r'[^\|]+(?=\|\|)'

        # 1. heading
        if line.startswith(re.match(pattern_heading, line):
            re.split(r'[\|\|]+', '|heading 1|heading 2||heading 3||')
            heading_match = re.match(pattern_heading, line)
            heading_number, heading_text = heading_match.groups()
            para.style = f'Heading {heading_number}'
            para.add_run(text=heading_text)
        # 2. list
        elif re.match(pattern_list, line):
            list_match = re.match(pattern_list, line)
            list_type, list_text = list_match.groups()
            # issue 1: couldn't resolve the round and square bullets differently
            # issue 2: find a way of decorating nested lists with different point styles
            # issue 3: sublevel list have unwanted top space to their parent list
            max_list_level = 3
            style_name = 'List Bullet'
            if list_type[:max_list_level][-1] == '#':
                style_name = "List Number"
            para.style = f'{style_name} {str(min(len(list_type), max_list_level)) if len(list_type) > 1 else ""}'.strip()
            para.add_run(text = list_text)
        elif re.findall(table_header_pattern, line):
            header_texts = re.findall(table_header_pattern, line)
            n_cols = len(header_texts)

            # Create a table
            if not cur_table:
              cur_table = doc.add_table(rows=1, cols=n_cols)
              cur_table.style = 'Table Grid'

            # populate header row --------
            heading_cells = cur_table.rows[0].cells
            for i in range(len(heading_cells)):
                heading_cells[i].text = header_texts[i]


            # # Set alignment to cell
            # # table.cell(0, 0).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

            # # Add image to the first column
            # table.cell(0, 0).paragraphs[0].add_run().add_picture(LOGO_PATH, width = LOGO_WIDTH)

            # # Add the text lines to the cell
            # # line 1
            # para1 = table.cell(0, 1).paragraphs[0]
            # para1.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
            # para1.paragraph_format.space_after = Pt(0)
            # run1 = para1.add_run('Group Data Analytics Weekly Report')
            # run1.font.size = Pt(16)
            # # line 2
            # para2 = table.cell(0, 1).add_paragraph(f'Activities for the Week ({convert_date_to_weekly_report_format(REPORT_FROM)} - {convert_date_to_weekly_report_format(REPORT_TO)})')
            # run2 = para2.runs[0]
            # run2.font.color.rgb = DOC_PRIMARY_COLOR # Set orange color
            # run2.font.size = Pt(14) # Set font size to 12 points
            # run2.font.bold = True


        else:
            para.add_run(text=line)

    return doc

convert_jira_to_docx(sample_jira_text).save('output.docx')

In [ ]:

# # Set alignment to cell
# # table.cell(0, 0).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# # Add image to the first column
# table.cell(0, 0).paragraphs[0].add_run().add_picture(LOGO_PATH, width = LOGO_WIDTH)

# # Add the text lines to the cell
# # line 1
# para1 = table.cell(0, 1).paragraphs[0]
# para1.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
# para1.paragraph_format.space_after = Pt(0)
# run1 = para1.add_run('Group Data Analytics Weekly Report')
# run1.font.size = Pt(16)
# # line 2
# para2 = table.cell(0, 1).add_paragraph(f'Activities for the Week ({convert_date_to_weekly_report_format(REPORT_FROM)} - {convert_date_to_weekly_report_format(REPORT_TO)})')
# run2 = para2.runs[0]
# run2.font.color.rgb = DOC_PRIMARY_COLOR # Set orange color
# run2.font.size = Pt(14) # Set font size to 12 points
# run2.font.bold = True

In [ ]:
# apply text efffects here


In [ ]:
# Initialize a new Document
doc = Document()

def convert_jira_to_docx(jira_text):

    # Split the Jira text into separate lines
    lines = jira_text.split('\n')

    # # Track if we are currently inside a table
    # inside_table = False
    # table = None
    # table_rows = []

    # Process each line of Jira text
    for line_pos, line in enumerate(lines):
        # if it's the first line take the existing paragraph, otherwise, create new one
        para = doc.paragraphs[line_pos] if line_pos == 0 else doc.add_paragraph()
        # Skip empty lines
        if not line.strip():
            para.add_run()
            continue

        # Check for different Jira elements
        if line.startswith("h1."):
            # Heading 1
            # heading_text = line.replace("h1.", "").strip()
            para.add_run(text=line, style=f'Heading {line_pos}')
            # doc.add_heading(heading_text, level=1)

        elif line.startswith("h2."):
            # Heading 2
            heading_text = line.replace("h2.", "").strip()
            doc.add_heading(heading_text, level=2)

        elif line.startswith("h3."):
            # Heading 3
            heading_text = line.replace("h3.", "").strip()
            doc.add_heading(heading_text, level=3)

        # elif line.startswith("|"):
        #     # Table row
        #     cells = [cell.strip() for cell in line.split("|")[1:-1]]
        #     table_rows.append(cells)

        #     # Check if it's the start of a new table
        #     if not inside_table:
        #         table = doc.add_table(rows=0, cols=len(cells))
        #         table.autofit = False
        #         inside_table = True

        # elif inside_table and line.startswith("||"):
        #     # End of table
        #     for row in table_rows:
        #         table.add_row().cells = [cell for cell in row]
        #     inside_table = False
        #     table_rows = []

        # else:
        #     # Normal paragraph
        #     if inside_table:
        #         for row in table_rows:
        #             table.add_row().cells = [cell for cell in row]
        #         inside_table = False
        #         table_rows = []

        #     p = doc.add_paragraph()
        #     p.add_run(line)

    return doc
